In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


### 군집화

In [ ]:
import pandas as pd
import warnings
warnings.filterwarnings('ignore')

# master_c0 = pd.read_excel('master_c0.xlsx', index_col=0)
# master_c1 = pd.read_excel('master_c1.xlsx', index_col=0)
# master_c2 = pd.read_excel('master_c2.xlsx', index_col=0)

master_c0 = pd.read_excel('/content/drive/Othercomputers/내 컴퓨터/hmkd1/lmembers 분석/master_c0.xlsx')
master_c1 = pd.read_excel('/content/drive/Othercomputers/내 컴퓨터/hmkd1/lmembers 분석/master_c1.xlsx')
master_c2 = pd.read_excel('/content/drive/Othercomputers/내 컴퓨터/hmkd1/lmembers 분석/master_c2.xlsx')
master_c0.head(2)

# 서프라이즈

In [2]:
! pip install scikit-surprise

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 772.0/772.0 kB 39.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for scikit-surprise: filename=scikit_surprise-1.1.3-cp310-cp310-linux_x86_64.whl size=3096326 sha256=e8611436b4ecfd54ec6f3cfab6e6b1678eef030c78ea47ba32889558cdfcc613
  Stored in directory: /root/.cache/pip/wheels/a5/ca/a8/4e28def53797fdc4363ca4af740db15a9c2f1595ebc51fb445
Successfully built scikit-surprise


In [ ]:
import surprise
print(surprise.__version__)

1.1.3


In [ ]:
from surprise import SVD
from surprise import Dataset
from surprise import accuracy
from surprise.model_selection import train_test_split

In [ ]:
# 중분류로 할 경우
import pandas as pd

df_ori = pd.read_pickle('/content/drive/Othercomputers/내 컴퓨터/hmkd1/lmembers 분석/surprise.pkl')
df = df_ori.copy()
df.drop(['총구매금액', '평균가'], axis=1, inplace=True)
df

,고객번호,중분류명,개수
0,00001,가공식품,157
1,00001,가루,1
2,00001,가정잡화,1
3,00001,간식,1
4,00001,간편조리,2
...,...,...,...
3037414,19383,핸드/풋케어,8
3037415,19383,헤어케어,12
3037416,19383,화장지/티슈,5
3037417,19383,화장품,1


In [ ]:
# 소분류로 할 경우
import pandas as pd

df_ori = pd.read_pickle('/content/drive/Othercomputers/내 컴퓨터/hmkd1/lmembers 분석/surprise_s.pkl')

In [ ]:
df = df_ori.copy()
df.고객번호 = df.고객번호.astype(int)
df.head(2)

,고객번호,중분류명,개수,총구매금액,평균가
0,1,가공식품,157,2750580,17519.617834
1,1,가루,1,2950,2950.000000


In [ ]:
x = master_c0['고객번호'].tolist()
y = master_c1['고객번호'].tolist()
z = master_c2['고객번호'].tolist()

In [ ]:
df_0=df[df['고객번호'].isin(x)]
df_1=df[df['고객번호'].isin(y)]
df_2=df[df['고객번호'].isin(z)]

In [ ]:
from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler()
scaled_count = scaler.fit_transform(df_0[['개수']])
scaled_mean_price = scaler.fit_transform(df_0[['평균가']])

df_0['개수_scaled'] = scaled_count
df_0['평균가_scaled'] = scaled_mean_price
df_0['합계'] = df_0['개수_scaled'] + df_0['평균가_scaled']

df_0 = df_0[['고객번호', '중분류명', '개수']]
df_0.head(2)

<ipython-input-77-4224c8be349c>:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_0['개수_scaled'] = scaled_count
<ipython-input-77-4224c8be349c>:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_0['평균가_scaled'] = scaled_mean_price
<ipython-input-77-4224c8be349c>:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/

,고객번호,중분류명,개수
611,7,LOCAL잎채소,1
612,7,NB구두,1


In [ ]:
import pandas as pd
from surprise import SVD
from surprise import SVDpp
from surprise import KNNBasic
from surprise import KNNBaseline
from surprise import Dataset
from surprise import Reader
from surprise.model_selection import train_test_split
from surprise import accuracy
from surprise import get_dataset_dir

def recomm_product_by_surprise(algo, userId, df_for_surprise, top_n=10):
    # Surprise 데이터셋 생성을 위한 Reader 객체 생성
    reader = Reader(rating_scale=(0, df_for_surprise['개수'].max()))

    # Surprise 데이터셋 생성
    data = Dataset.load_from_df(df_for_surprise[['고객번호', '중분류명', '개수']], reader)

    # 데이터셋을 학습용과 테스트용으로 분할
    trainset, _ = train_test_split(data, test_size=0.25, random_state=42)

    # 학습 데이터셋으로 알고리즘 학습
    algo.fit(trainset)

    # 특정 사용자에게 상품 추천
    testset = trainset.build_anti_testset()
    testset = [t for t in testset if t[0] == userId]
    predictions = algo.test(testset)

    # 예측 결과 정렬하여 상위 상품 추출
    top_predictions = sorted(predictions, key=lambda x: x.est, reverse=True)[:top_n]

    # 추천 상품 정보 출력
    recommended_products = []
    for prediction in top_predictions:
        recommended_products.append(prediction[1])

    return recommended_products


In [ ]:
# algo = SVD(n_epochs=20, random_state=0)
algo = SVD(random_state=0)
recomm_product_by_surprise(algo, 7, df_0, top_n=10)

['즉석식품', '냉동만두', '두부', '안전용품', '흰우유', '위생용품', '단무지류', '편의시설', '브랜드빵', '헬스용품']

In [ ]:
algo = KNNBasic(random_state=0)
recomm_product_by_surprise(algo, 7, df_0, top_n=10)

Computing the msd similarity matrix...
Done computing similarity matrix.


['과자', '주유소', '흰우유', '음료', '스낵', '주류', '구강케어', '라면', '문구/팬시', '여성용품']

In [ ]:
algo = SVDpp(random_state=0)
recomm_product_by_surprise(algo, 7, df_0, top_n=10)

['즉석식품', '냉동만두', '두부', '안전용품', '흰우유', '위생용품', '단무지류', '편의시설', '브랜드빵', '헬스용품']

In [ ]:
algo = KNNBaseline(random_state=0)
recomm_product_by_surprise(algo, 7, df_0, top_n=10)

Estimating biases using als...
Computing the msd similarity matrix...
Done computing similarity matrix.


['과자', '주유소', '흰우유', '주류', '스낵', '음료', '라면', '문구/팬시', '구강케어', '비스켓']

In [ ]:
df[df.고객번호 == 7].sort_values(by='합계', ascending=False)

,고객번호,소분류명,합계
1448,00007,청과,0.048385
1484,00007,한식델리,0.045557
1327,00007,브랑제리,0.044135
1366,00007,숙녀고정행사,0.039713
1372,00007,시니어,0.032240
...,...,...,...
1303,00007,마시는요구르트,0.000014
1490,00007,혼합탄산,0.000014
1277,00007,냉장면,0.000014
1364,00007,수입콩나물,0.000014


In [ ]:
from surprise.model_selection import GridSearchCV

reader = Reader(rating_scale=(0, df_0['개수'].max()))
data = Dataset.load_from_df(df_0[['고객번호', '중분류명', '개수']], reader)

# 최적화할 파라미터를 딕셔너리 형태로 지정.
param_grid = {'n_epochs': [20, 40, 60], 'n_factors': [50, 100, 200] }

# CV를 3개 폴드 세트로 지정, 성능 평가는 rmse, mse로 수행하도록 GridSearchCV 구성
gs = GridSearchCV(SVDpp, param_grid, measures=['rmse', 'mae'], cv=3)
gs.fit(data)

# 최고 RMSE Evaluation 점수와 그때의 하이퍼 파라미터
print(gs.best_score['rmse'])
print(gs.best_params['rmse'])

In [ ]:
# 8분 24초

import pandas as pd
from surprise import Reader, Dataset
from surprise.model_selection import cross_validate

# reader = Reader(rating_scale=(0.5, 5.0))
reader = Reader(rating_scale=(0, 1))
data = Dataset.load_from_df(df[['고객번호', '소분류명', '합계']], reader)

algo = SVD(random_state=0)
cross_validate(algo, data, measures=['RMSE', 'MAE'], cv=5, verbose=True)

Evaluating RMSE, MAE of algorithm SVD on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    0.0145  0.0145  0.0145  0.0145  0.0147  0.0145  0.0001  
MAE (testset)     0.0062  0.0062  0.0061  0.0061  0.0062  0.0062  0.0000  
Fit time          65.70   68.65   69.44   68.79   68.74   68.26   1.31    
Test time         21.87   17.73   16.60   17.81   20.33   18.87   1.93    


{'test_rmse': array([0.01445143, 0.01447852, 0.01449379, 0.0144612 , 0.01470431]),
 'test_mae': array([0.00616803, 0.00617471, 0.00614268, 0.00614975, 0.00617552]),
 'fit_time': (65.70393109321594,
  68.65142488479614,
  69.44053411483765,
  68.78543329238892,
  68.73663663864136),
 'test_time': (21.87056851387024,
  17.73251175880432,
  16.603623390197754,
  17.810500621795654,
  20.325324296951294)}

In [ ]:
# # DatasetAutoFolds - 데이터 세트 전체를 학습 데이터로 사용하게 해 줌
# # build_full_trainset() - 위의 클래스로 객체를 생성한 뒤, 이 메서드를 호출하면 전체 데이터를
# # 학습 데이터 세트로 만들 수 있음

# from surprise.dataset import DatasetAutoFolds

# reader = Reader(line_format='user item rating timestamp', sep=',', rating_scale=(0.5, 5))
# # DatasetAutoFolds 클래스를 ratings_noh.csv 파일 기반으로 생성.
# df_for_surprise.to_csv('/content/drive/Othercomputers/내 컴퓨터/hmkd1/dataset/ml-latest-small/df_for_surprise.csv', index=False, header=False)
# data_folds = DatasetAutoFolds(ratings_file='/content/drive/Othercomputers/내 컴퓨터/hmkd1/dataset/ml-latest-small/df_for_surprise.csv', reader=reader)

# #전체 데이터를 학습데이터로 생성함.
# trainset = data_folds.build_full_trainset()

In [ ]:
# # 추천 대상 영화 list
# # [Prediction(uid='9', iid='1', r_ui=None, est=3.6398015009715396, details={'was_impossible': False}),
# #  Prediction(uid='9', iid='2', r_ui=None, est=3.0708535569743463, details={'was_impossible': False}),...]

# def recomm_movie_by_surprise(algo, userId, unseen_movies, top_n=10):
#     # 알고리즘 객체의 predict() 메서드를 평점이 없는 영화에 반복 수행한 후 결과를 list 객체로 저장
#     predictions = [algo.predict(str(userId), str(movieId)) for movieId in unseen_movies]
#     # predictions list 객체는 surprise의 Predictions 객체를 원소로 가지고 있음.
#     # [Prediction(uid='9', iid='1', est=3.69), Prediction(uid='9', iid='2', est=2.98),,,,]
#     # 이를 est 값으로 정렬하기 위해서 아래의 sortkey_est 함수를 정의함.
#     # sortkey_est 함수는 list 객체의 sort() 함수의 키 값으로 사용되어 정렬 수행.
#     def sortkey_est(pred):
#         return pred.est

#     # sortkey_est( ) 반환값의 내림 차순으로 정렬 수행하고 top_n개의 최상위 값 추출.
#     predictions.sort(key=sortkey_est, reverse=True)    # 내림차순임
#     top_predictions= predictions[:top_n]

#     # top_n으로 추출된 영화의 정보 추출. 영화 아이디, 추천 예상 평점, 제목 추출
#     top_movie_ids = [ int(pred.iid) for pred in top_predictions]
#     top_movie_rating = [ pred.est for pred in top_predictions]
#     top_movie_titles = movies[movies.movieId.isin(top_movie_ids)]['title']
#     top_movie_preds = [ (id, title, rating) for id, title, rating in zip(top_movie_ids, top_movie_titles, top_movie_rating)]

#     return top_movie_preds

# unseen_movies = get_unseen_surprise(ratings, movies, 9)
# top_movie_preds = recomm_movie_by_surprise(algo, 9, unseen_movies, top_n=10)
# print('##### Top-10 추천 영화 리스트 #####')

# for top_movie in top_movie_preds:
#     print(top_movie[1], ":", top_movie[2])

NameError: ignored

In [ ]:
recomm_product_by_surprise(algo, '00007', df, top_n=10)

['유아브랜드', '액상조미료', '전통과자', '사과', '간편조리행사', '기타의류', '대중선어', '잼', '식기', '마른안주']

In [ ]:
recomm_product_by_surprise(algo, '00007', df, top_n=10)

Computing the msd similarity matrix...
Done computing similarity matrix.


['과자', '음료', '주유소', '기초화장품', '화장품브랜드샵', '아동', '푸드코트', '애견용품', '국내화장품', '편의시설']

NameError: ignored

Evaluating RMSE, MAE of algorithm SVD on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    23.1904 23.3987 23.2274 23.1508 23.2154 23.2365 0.0852  
MAE (testset)     8.4300  8.4087  8.4129  8.4108  8.4055  8.4136  0.0086  
Fit time          33.05   31.88   32.06   30.03   31.93   31.79   0.98    
Test time         11.33   5.87    6.89    7.79    5.95    7.56    2.01    


{'test_rmse': array([23.19035253, 23.39872752, 23.22740574, 23.15077044, 23.21538997]),
 'test_mae': array([8.43003437, 8.40871035, 8.41294092, 8.41082234, 8.4054632 ]),
 'fit_time': (33.0541250705719,
  31.881619215011597,
  32.06323289871216,
  30.025591373443604,
  31.92580008506775),
 'test_time': (11.333791255950928,
  5.865387439727783,
  6.885962963104248,
  7.788859844207764,
  5.945485830307007)}

In [ ]:
from surprise import KNNBasic

# KNNBasic 알고리즘 인스턴스 생성
knn_algo = KNNBasic(k=50, min_k=5, sim_options={'name': 'pearson', 'user_based': True}, verbose=True)


서프라이즈(Surprise)는 추천 시스템을 위한 파이썬 라이브러리로, KNNBasic은 Surprise 패키지의 K-최근접 이웃(K-Nearest Neighbors) 알고리즘 중 하나입니다. KNNBasic 알고리즘을 사용할 때 주요한 매개변수들은 다음과 같습니다:

k (기본값: 40): 최근접 이웃의 개수를 의미합니다. 이웃의 개수가 많을수록 모델의 복잡도가 증가하고 실행 시간이 더 오래 걸릴 수 있습니다. 적절한 k 값을 설정하는 것은 모델의 성능에 영향을 미칩니다.

min_k (기본값: 1): 예측을 수행하기 위해 최소한 필요한 이웃의 개수입니다. min_k 이하의 이웃을 가진 사용자나 상품에 대해서는 예측을 수행하지 않습니다.

sim_options: 유사도 계산 방법을 설정하는 옵션입니다. 주요 매개변수로는 name과 user_based가 있습니다. name은 유사도 계산 방법을 선택하며, "cosine", "pearson", "pearson_baseline", "msd", "cosine_baseline" 등이 사용될 수 있습니다. user_based는 유사도를 계산하는 방향을 의미하며, True일 경우 사용자 기반 유사도를, False일 경우 상품 기반 유사도를 사용합니다.

verbose (기본값: True): 알고리즘의 실행 과정을 출력할지 여부를 설정합니다. True로 설정하면 실행 과정이 출력되고, False로 설정하면 출력이 생략됩니다.

이러한 매개변수들을 조정하여 KNNBasic 알고리즘을 설정할 수 있습니다. 예를 들어, 아래는 KNNBasic 알고리즘의 매개변수 설정 예시입니다:

UserId - 고객ID
ItemId - 상품분류체계(대분류, 유형, 고중저가 등의 형태)
ratings - 구매횟수(빈도), 구매금액 -> 지수화

In [ ]:
# # 기존 master_c0 등을 서프라이즈에 맞게끔 가공하는 코드

# import pandas as pd
# from surprise import Dataset
# from surprise import Reader

# # 건수 컬럼들을 하나의 컬럼으로 합치기
# master_c0_f['Total_count'] = master_c0_f.sum(axis=1)

# # 데이터프레임을 Surprise 라이브러리의 Dataset 형식으로 변환
# reader = Reader(rating_scale=(0, 1))  # rating_scale은 예시로 설정한 범위에 맞게 조정해주세요

# # 데이터프레임에서 사용자 ID, 아이템 ID, 평점 컬럼으로 구성된 새로운 데이터프레임 생성
# df_for_surprise = pd.DataFrame({'고객번호': master_c0_f.index.repeat(len(master_c0_f.columns)),
#                                '상품': master_c0_f.columns.tolist() * len(master_c0_f),
#                                '횟수': master_c0_f.values.flatten()})

# dataset = Dataset.load_from_df(df_for_surprise, reader)

# # 결과 확인
# df_for_surprise


,고객번호,상품,횟수
0,7,가공건수,312
1,7,신선건수,137
2,7,외식/편의시설건수,0
3,7,일상건수,68
4,7,의료건수,2
...,...,...,...
49585,19325,스포츠건수,20
49586,19325,패션/잡화건수,10
49587,19325,유아/아동건수,5
49588,19325,명품건수,10
